In [1]:


from core.training.make import init_from_config_file

trainer, train_state = init_from_config_file('../example_configs/test1.yaml', debug=True)


In [ ]:
# train_state = trainer.swap_states(train_state, get_train=False)
trainer.train_epoch(train_state)